シーケンスデータの作成（キャリアの状態を文字列に変換）

シーケンス分析の準備

In [ ]:
import pandas as pd

# 1. CSVを読み込む
df = pd.read_csv("https://github.com/naoki-kokaze/British_Warship_Career/tree/main/data/query4sequence.csv")

# 2. 日付の正規化 (この段階ではまだ文字列かNaN)
df['startYear'] = df['startDate'].str.extract(r'(\d{4})')
df['endYear'] = df['endDate'].str.extract(r'(\d{4})')

# 3. 日付情報が欠損している行を削除 (スキップ)
#    'startYear' または 'endYear' のどちらかがNaNの行は、分析から除外する
original_count = len(df)
df = df.dropna(subset=['startYear', 'endYear'])
new_count = len(df)

if original_count > new_count:
    print(f"日付情報が欠損していた {original_count - new_count} 件の Presence レコードを分析から除外しました。")

# 4. 欠損行がなくなったため、安全に整数 (integer) に変換できる
df['startYear'] = df['startYear'].astype(int)
df['endYear'] = df['endYear'].astype(int)

# 5. 分析に不要な列を削除 (元のコードの続き)
df = df[['presence', 'shipLabel', 'statusLabel', 'startYear', 'endYear']]

# 6. 分析の全体期間を決定
min_year = df['startYear'].min()
max_year = df['endYear'].max()
print(f"分析対象期間: {min_year}年から{max_year}年まで")

# 艦種コードを 'presence' URI から抽出
# 例: '.../presence_SL3_0056_02' -> 'SL3'
# 正規表現 'presence_([^_]+)' を使用
# （もし 'presence_' で始まらないURIがある場合は 'UNK' (不明) とする）
df['shipType'] = df['presence'].str.extract(r'presence_([^_]+)').fillna('UNK')

In [ ]:
# 例：状態を単純化するマッピング
status_map = {
    'warship#InCommission': 'C',  # 現役
    'warship#InOrdinary': 'O',    # 予備役
    'warship#PaidOff': 'P',    # PaidOff
    'warship#InNonCombatDuty': 'N',  # 非戦闘任務
    'warship#UnderRepair': 'R',    # 修理中
    'warship#UnderFitting': 'F'   # 艤装中
}
# マッピングにないものは 'Other' (その他) にする
df['status'] = df['statusLabel'].map(status_map).fillna('Other')

In [ ]:
all_sequences = {}
all_ships = df['shipLabel'].unique()

for ship in all_ships:
    ship_data = df[df['shipLabel'] == ship].sort_values(by='startYear')

    # その艦のキャリア開始年と終了年
    ship_start = ship_data['startYear'].min()
    ship_end = ship_data['endYear'].max()

    ship_sequence = []
    for year in range(ship_start, ship_end + 1):
        # その年に該当する状態を探す
        # (year が startYear と endYear の間にあるレコード)
        current_status = 'None' # キャリアの隙間は 'None'

        # シンプルな割り当てロジック
        for _, period in ship_data.iterrows():
            if period['startYear'] <= year and period['endYear'] >= year:
                current_status = period['status']
                break # 最初に見つかったものを採用

        ship_sequence.append(current_status)

    # 艦名をキーとして、状態のリスト（シーケンス）を保存
    all_sequences[ship] = ship_sequence

# これで all_sequences に {'HMS Trafalgar': ['O', 'O', 'C', 'C', 'H', ...], ...} のようなデータが完成

In [ ]:
# --- 各艦の「絶対的なキャリア開始年」を取得 ---
# 元の 'df' から、各艦の最初の 'startYear' を辞書として取得
ship_start_years = df.groupby('shipLabel')['startYear'].min().to_dict()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# --- 準備 ---
# 以下の変数がメモリに存在することを前提とする
# all_sequences: 辞書 {'HMS Trafalgar': ['O', 'O', 'C',...], ...} (元の艦名がキー)
# df: DataFrame (SPARQLから読み込んだ全データ、'shipLabel', 'startYear' を含む)
# ship_start_years: 辞書 {'HMS Trafalgar': 1841, ...} (各艦のキャリア開始西暦)
# ---------------------------------------------------------------------

# --- 0. 状態と色の定義 ---
states_to_plot = ['C', 'F', 'O', 'R', 'P', 'N', 'Other']
colors = {
    'C': '#4b4b4b',  # 現役
    'P': '#d9d9d9', # 乗員解除
    'O': '#E07B39',   # 予備役
    'F': '#7a7a7a', # 艤装中
    'N': '#3b8ea5',  # 非戦闘
    'R': '#b3b3b3',     # 修理中
    'Other': '#ffffff' # その他
}
plot_colors = [colors[s] for s in states_to_plot]

try:
    # --- 1. 全体の時間範囲を決定 ---
    global_min_year = df['startYear'].min()
    global_max_year = df['endYear'].max()
    years_index = range(global_min_year, global_max_year + 1)
    stock_data = pd.DataFrame(0, index=years_index, columns=states_to_plot + ['None'])
    total_years = len(years_index) # 全年数を計算

    # --- 2. 全艦船のシーケンスを絶対時間（西暦）に配置し直し、集計 ---
    num_ships_processed = 0
    for ship_key, sequence in all_sequences.items():
        if ship_key not in ship_start_years:
            continue

        start_year = ship_start_years[ship_key]
        num_ships_processed += 1

        for i, state in enumerate(sequence):
            current_year = start_year + i
            if state in stock_data.columns:
                if current_year in stock_data.index:
                    stock_data.loc[current_year, state] += 1

    print(f"集計完了: {num_ships_processed}隻のキャリアを西暦 {global_min_year}年～{global_max_year}年 のタイムラインに配置しました。")

    # --- 3. 積層グラフの描画 ---
    plt.figure(figsize=(20, 10))
    ax = plt.gca() # <-- 変更点：軸(ax)を取得

    plot_values = stock_data[states_to_plot].T

    plt.stackplot(
        stock_data.index,  # X軸: 西暦年
        plot_values,       # Y軸: 各状態の隻数 (絶対数)
        labels=states_to_plot,
        colors=plot_colors
    )

    # --- 4. グラフの装飾 ---
    title = f'Fleet Status Stock Plot (All {num_ships_processed} Sampled Ships) [10-Year Ticks]'
    filename = "/content/drive/MyDrive/PhD Thesis/Winfield RDF/data_analysis/all_ships_absolute_stock_plot_10yr.svg"

    plt.title(title, fontsize=16)
    plt.xlabel('Year (Absolute Time)', fontsize=12)
    plt.ylabel('Number of Ships (Absolute Count)', fontsize=12)
    plt.legend(loc='upper left')
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    # SVG（ベクタ形式）保存
    plt.rcParams['svg.fonttype'] = 'none'  # フォントをアウトライン化せず保持
    plt.savefig(filename, format='svg', bbox_inches='tight')
    print(f"SVG plot saved to {filename}")

    tick_step = 10 # 10年ごとに目盛りを打つ

    # 最初の目盛り (例: 1817年 -> 1820年)
    start_tick_year = (global_min_year // tick_step + 1) * tick_step

    # X軸の目盛りラベルを作成 (例: 1820, 1830...)
    tick_labels = np.arange(start_tick_year, global_max_year + 1, tick_step)

    # 目盛りの位置は、ラベル（西暦）そのものを指定できる
    ax.set_xticks(tick_labels)
    ax.set_xticklabels(tick_labels, rotation=45, ha='right')

    # X軸の表示範囲を、データが実際に存在する範囲に限定
    plt.xlim(global_min_year, global_max_year)

    plt.tight_layout()
    plt.savefig(filename)

    print(f"艦隊ストック積層グラフ（10年刻み）を '{filename}' として保存しました。")

except NameError as e:
    print(f"エラー: 必要な変数 (df, all_sequences, ship_start_years) が見つかりません。 {e}")
except Exception as e:
    print(f"予期せぬエラーが発生しました: {e}")